<a href="https://colab.research.google.com/github/kabirNahata/Learning-journey/blob/main/pdf_chatbot_LLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Implementing RAG

In [60]:
from langchain.document_loaders import PyPDFLoader

In [61]:
pdf_obj = PyPDFLoader("CV_AnishChapagain_2024.pdf")
pdf_doc = pdf_obj.load()

In [ ]:
pdf_doc

In [ ]:
dir(pdf_doc)

In [64]:
pdf_doc[0].metadata

{'producer': 'Microsoft® Word 2016',
 'creator': 'Microsoft® Word 2016',
 'creationdate': '2024-06-10T16:37:00+05:45',
 'title': 'Anish Chapagain:CV',
 'author': 'PETERCHAPAGAIN',
 'moddate': '2024-06-10T16:37:00+05:45',
 'source': 'CV_AnishChapagain_2024.pdf',
 'total_pages': 4,
 'page': 0,
 'page_label': '1'}

In [65]:
pdf_doc[-1].metadata

{'producer': 'Microsoft® Word 2016',
 'creator': 'Microsoft® Word 2016',
 'creationdate': '2024-06-10T16:37:00+05:45',
 'title': 'Anish Chapagain:CV',
 'author': 'PETERCHAPAGAIN',
 'moddate': '2024-06-10T16:37:00+05:45',
 'source': 'CV_AnishChapagain_2024.pdf',
 'total_pages': 4,
 'page': 3,
 'page_label': '4'}

In [66]:
len(pdf_doc)

4

In [67]:
from langchain.text_splitter import CharacterTextSplitter, RecursiveCharacterTextSplitter

In [68]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1000,
    chunk_overlap = 100
)

In [69]:
texts = text_splitter.split_documents(pdf_doc)

In [70]:
len(texts)

14

In [71]:
texts[0].metadata

{'producer': 'Microsoft® Word 2016',
 'creator': 'Microsoft® Word 2016',
 'creationdate': '2024-06-10T16:37:00+05:45',
 'title': 'Anish Chapagain:CV',
 'author': 'PETERCHAPAGAIN',
 'moddate': '2024-06-10T16:37:00+05:45',
 'source': 'CV_AnishChapagain_2024.pdf',
 'total_pages': 4,
 'page': 0,
 'page_label': '1'}

In [72]:
texts[-1].metadata

{'producer': 'Microsoft® Word 2016',
 'creator': 'Microsoft® Word 2016',
 'creationdate': '2024-06-10T16:37:00+05:45',
 'title': 'Anish Chapagain:CV',
 'author': 'PETERCHAPAGAIN',
 'moddate': '2024-06-10T16:37:00+05:45',
 'source': 'CV_AnishChapagain_2024.pdf',
 'total_pages': 4,
 'page': 3,
 'page_label': '4'}

In [ ]:
!pip install langchain_huggingface

In [74]:
from langchain_huggingface import HuggingFaceEmbeddings

In [75]:
embeddings_model = HuggingFaceEmbeddings(model_name='sentence-transformers/all-mpnet-base-v2') # uses default or provide model name

In [76]:
embeddings_model

HuggingFaceEmbeddings(model_name='sentence-transformers/all-mpnet-base-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, query_encode_kwargs={}, multi_process=False, show_progress=False)

In [77]:
embeddings = embeddings_model.embed_documents([t.page_content for t in texts])

In [ ]:
embeddings[2]

In [79]:
print(f"Chunk size: {len(embeddings[0])}")
print(f"Shape: {(len(embeddings), len(embeddings[0]))}")

Chunk size: 768
Shape: (14, 768)


In [80]:
from langchain.vectorstores import Chroma

In [ ]:
!pip install chromadb

In [83]:
persist_directory = 'sample_data/chroma'

vectordb = Chroma.from_documents(
    documents=texts,
    embedding=embeddings_model,
    persist_directory=persist_directory
)

In [84]:
vectordb.persist()

<ipython-input-84-d832e715b29a>:1: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vectordb.persist()


In [85]:
print(vectordb._collection.count())

14


In [86]:
print(dir(vectordb))

['_Chroma__query_collection', '_LANGCHAIN_DEFAULT_COLLECTION_NAME', '__abstractmethods__', '__annotations__', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__len__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__slots__', '__str__', '__subclasshook__', '__weakref__', '_abc_impl', '_asimilarity_search_with_relevance_scores', '_client', '_client_settings', '_collection', '_cosine_relevance_score_fn', '_embedding_function', '_euclidean_relevance_score_fn', '_get_retriever_tags', '_max_inner_product_relevance_score_fn', '_persist_directory', '_select_relevance_score_fn', '_similarity_search_with_relevance_scores', 'aadd_documents', 'aadd_texts', 'add_documents', 'add_images', 'add_texts', 'adelete', 'afrom_documents', 'afrom_texts', 'aget_by_ids', 'amax_marginal_relev

In [87]:
docs = vectordb.similarity_search("List all the links available", k=3)

In [88]:
docs

[Document(metadata={'total_pages': 4, 'creator': 'Microsoft® Word 2016', 'creationdate': '2024-06-10T16:37:00+05:45', 'author': 'PETERCHAPAGAIN', 'page': 3, 'source': 'CV_AnishChapagain_2024.pdf', 'moddate': '2024-06-10T16:37:00+05:45', 'producer': 'Microsoft® Word 2016', 'title': 'Anish Chapagain:CV', 'page_label': '4'}, page_content='● Freelancing, Web development using CakePHP (www.netrenos.com) - 2011  \n● Library Management System using Oracle with Developer, 2004 (Himalayan IT)  \n   \n \nLinkedIn Profile: https://www.linkedin.com/in/anish-chapagain-a581386/ \nReferences are available upon request and as required.'),
 Document(metadata={'source': 'CV_AnishChapagain_2024.pdf', 'author': 'PETERCHAPAGAIN', 'creator': 'Microsoft® Word 2016', 'page': 2, 'page_label': '3', 'moddate': '2024-06-10T16:37:00+05:45', 'producer': 'Microsoft® Word 2016', 'total_pages': 4, 'creationdate': '2024-06-10T16:37:00+05:45', 'title': 'Anish Chapagain:CV'}, page_content='●  Lecturing in Programming Lan

In [ ]:
for doc in docs:
    print(doc.page_content)
    print(f"----" *20)

In [ ]:
search_Web = vectordb.similarity_search("how often the term 'Web' is mentioned", k=3)

In [ ]:
for web in search_Web:
    print(web.page_content[:-1])

In [ ]:
retriever = vectordb.as_retriever()

In [ ]:
doocs = retriever.invoke("web scrapping")
doocs

In [ ]:
for dooc in doocs:
  print(dooc.page_content)

In [ ]:
# prompt: pip install with verbose false

!pip install --quiet langchain_huggingface

In [ ]:
!pip install langchain_google_genai

In [90]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.schema import StrOutputParser

In [131]:
key=""

In [91]:
# llm_model = ChatGoogleGenerativeAI(
#     model="gemini-1.5-flash-latest",
#     google_api_key=api_key,
#     )
llm_model = ChatGoogleGenerativeAI(
     model="gemini-2.0-flash",
     google_api_key=key,
     )

In [92]:
llm_model

ChatGoogleGenerativeAI(model='models/gemini-2.0-flash', google_api_key=SecretStr('**********'), client=<google.ai.generativelanguage_v1beta.services.generative_service.client.GenerativeServiceClient object at 0x7c923aa1a8d0>, default_metadata=(), model_kwargs={})

In [93]:
from langchain import PromptTemplate

In [94]:
llm_prompt_template = """You are an assistant, use the following context to answer the question.
Use two to three sentences maximum and keep the answer concise.\n
Question: {question} \nContext: {context} \nAnswer:"""

In [95]:
llm_prompt = PromptTemplate.from_template(llm_prompt_template)
print(llm_prompt)

input_variables=['context', 'question'] input_types={} partial_variables={} template='You are an assistant, use the following context to answer the question.\nUse two to three sentences maximum and keep the answer concise.\n\nQuestion: {question} \nContext: {context} \nAnswer:'


In [96]:
from langchain.schema.runnable import RunnablePassthrough

In [99]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

def reply(question=''):
    chain = (
    {"context": vectordb.as_retriever() | format_docs, "question": RunnablePassthrough()}
    | llm_prompt | llm_model | StrOutputParser()
    )
    print(chain.invoke(question))

In [100]:
reply("how often the term 'Web' is mentioned")

The term "Web" is mentioned 11 times in the provided context. It appears in the context of web scraping, web development, and website-related projects.


In [124]:
reply("How many programming languages did you find in the context.")

I found 14 programming languages in the context. These include C, C++, Java, PL/SQL, HTML, PHP, JavaScript, CSS, Python, Shell Scripts, Django, PySpark, PyTorch, and Pandas.


In [125]:
reply("How many programming languages did you find in the context, provide me with the lists of the languages.")

I found 20 programming languages in the context. The languages are C, C++, Java, Oracle with Developer & PL/SQL, Borland C++ Builder, HTML, PHP, MySQL, JavaScript, CSS, Python, Shell Scripts, .NET, LINQ, MVC and others like NLTK, Scikit-learn, PySpark, PyTorch, Django, Pandas.


In [126]:
reply("How many programming languages did you find in the context.")

Based on the context, I found 14 programming languages: C, C++, Java, Oracle with Developer & PL/SQL, Borland C++ Builder, HTML, PHP, JavaScript, Python, Shell Scripts, SQL, PySpark, PyTorch, and Django.


In [128]:
reply("How many programming languages did you find in the context, provide me with the lists of the languages only.")

I found 19 programming languages in the context. They are: C, C++, Java, PL/SQL, HTML, PHP, JavaScript, CSS, Python, Shell Scripts, NLTK, PySpark, PyTorch, Django, .NET, LINQ, MVC, MySQL, and Terraform.


In [129]:
reply("How many programming languages did you find in the context, provide me with the lists of the languages.Give them in bullet points")

I found 18 programming languages in the context. Here is a list of them:

*   C
*   C++
*   Java
*   PL/SQL
*   HTML
*   PHP
*   MySQL
*   JavaScript
*   CSS
*   Python
*   Shell Scripts
*   NLTK
*   Django
*   Pandas
*   .NET
*   LINQ
*   MVC
*   PySpark
*   PyTorch


In [105]:
# search_Web = vectordb.similarity_search("how often the term 'Web' is mentioned", k=3)
search_Web

[Document(metadata={'creationdate': '2024-06-10T16:37:00+05:45', 'page': 2, 'page_label': '3', 'title': 'Anish Chapagain:CV', 'creator': 'Microsoft® Word 2016', 'author': 'PETERCHAPAGAIN', 'moddate': '2024-06-10T16:37:00+05:45', 'source': 'CV_AnishChapagain_2024.pdf', 'producer': 'Microsoft® Word 2016', 'total_pages': 4}, page_content='2nd Edition”. ISBN: 9781837636211, Oct-2023.  \n● “Hands-On Web Scraping with Python”. ISBN: 9781789533392, Jul-2019. \n \n \nEducation     \nM.Sc Computer System (2007-2008), Bangor University, Wales (U.K)  \n● Learning/Development using Java and Python (AI, Text Mining, Web Mining, NLP, Web Scraping). PHP/ \nJavaScript/ MySQL for Web development.   \n● Dissertation: Development of  Text Mining Toolkit ‘TUI’, a GUI application in Python handling Text Mining \nframework (TMT) in C, by Dr. WJ Teahan. \n● Development of Personal profiling system ‘webwiki’ using Python (Web Scraping techniques, Wikipedia).  \n  \nExecutive-MBA (HRM) (2004-2006), Himalayan W

In [119]:
search_mmr = vectordb.max_marginal_relevance_search("How many programming languages did you find in the context",k=3)
search_mmr

[Document(metadata={'author': 'PETERCHAPAGAIN', 'producer': 'Microsoft® Word 2016', 'moddate': '2024-06-10T16:37:00+05:45', 'page': 0, 'total_pages': 4, 'page_label': '1', 'title': 'Anish Chapagain:CV', 'creator': 'Microsoft® Word 2016', 'creationdate': '2024-06-10T16:37:00+05:45', 'source': 'CV_AnishChapagain_2024.pdf'}, page_content='\uf0b7 Data & Text Analysis, Regular Expressions, Analysis/Visualization, Reporting, Shell Scripting. \n\uf0b7 Integrating LLMs into application using APIs. Using Hugging Face Transformers for NLP tasks. \n\uf0b7 Experience using cloud platforms: GCP, AWS.  \n\uf0b7 Experienced with DBMS: Oracle, MySQL, MSSQL, MongoDB, and SQL Querying. \n\uf0b7 Experienced using Project Management applications BaseCamp, Blossom. Atlassian JIRA. \n\uf0b7 Proficiency in SDLC, Technical writing/support, Documentation and delivering presentations & articles.  \n \nProfessional Experiences  \nProfessional Freelancer.                              \nFreelance (Part-Time)      

In [121]:
def get_content(search_mmr):
  for doc in search_mmr:
      print(doc.page_content)
      print(f"----" *20)

In [122]:
get_content(search_mmr)

 Data & Text Analysis, Regular Expressions, Analysis/Visualization, Reporting, Shell Scripting. 
 Integrating LLMs into application using APIs. Using Hugging Face Transformers for NLP tasks. 
 Experience using cloud platforms: GCP, AWS.  
 Experienced with DBMS: Oracle, MySQL, MSSQL, MongoDB, and SQL Querying. 
 Experienced using Project Management applications BaseCamp, Blossom. Atlassian JIRA. 
 Proficiency in SDLC, Technical writing/support, Documentation and delivering presentations & articles.  
 
Professional Experiences  
Professional Freelancer.                              
Freelance (Part-Time)                                                       1st Apr, 2009 – till date.  
 Data extraction, automation, classifying, pattern recognition, analyzing, conducting EDA, prediction. 
 Track & Trace, Ecommerce: reviews, analysis, comparison, scheduling scrapper and reports. 
 Building applications for data collection, text summarization, translation, and paraphrasing.
-----